In [ ]:
 import os
working_path = '/content/drive/My Drive/s182190/data'
os.chdir(f'{working_path}')
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from geopy.distance import vincenty, geodesic


# Data Preparation, Geolife Style

In [ ]:
dataset_merged = pd.read_pickle(f'{working_path}/Geolife/dataset_merged.pkl')
dataset_merged.head()

,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count,mode,purpose,str_ts
0,1,2018-06-25 11:05:34+00:00,1483,1.394210e+06,7.516647e+06,1.394210e+06,7.516647e+06,1,3.0,0.0,1775.0,7.265090,-1.759773,0.004093,0.000002,1.299110e-09,2,10,0,6,-100,Informal,2018-06-25-11
1,1,2018-06-25 11:05:35+00:00,1484,1.394210e+06,7.516647e+06,1.394210e+06,7.516647e+06,1,3.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2,10,1,6,-100,Informal,2018-06-25-11
2,1,2018-06-25 11:06:16+00:00,1487,1.394210e+06,7.516643e+06,1.394210e+06,7.516643e+06,1,3.0,0.0,41.0,4.633838,-1.749762,0.113020,0.002757,6.723405e-05,2,10,2,6,-100,Informal,2018-06-25-11
3,1,2018-06-25 11:07:07+00:00,1488,1.394208e+06,7.516636e+06,1.394208e+06,7.516636e+06,1,3.0,0.0,51.0,6.745727,-1.754004,0.132269,0.002594,5.085319e-05,2,10,3,6,-100,Informal,2018-06-25-11
4,1,2018-06-25 11:07:41+00:00,1491,1.394211e+06,7.516648e+06,1.394211e+06,7.516648e+06,1,3.0,0.0,34.0,10.506570,1.359983,0.309017,0.009089,2.673155e-04,2,10,4,6,-100,Informal,2018-06-25-11


In [ ]:
!pip install pyproj
from pyproj import Proj, transform

     |████████████████████████████████| 6.5MB 5.4MB/s 


In [ ]:
wgs84= Proj("EPSG:4326")
etrs89_utm32= Proj("EPSG:25832")
web_mercator= Proj("EPSG:3857")

project = lambda x, y: transform(wgs84, etrs89_utm32, x, y)
inverse = lambda x, y: transform(etrs89_utm32, wgs84, x, y)
project_webM = lambda x, y: transform(wgs84, web_mercator, x, y)

In [ ]:
lon, lat = inverse(dataset_merged.x, dataset_merged.y)

dataset = pd.concat([dataset_merged.user, pd.DataFrame(np.c_[lon,lat], columns=["lon","lat"]), dataset_merged.ts,dataset_merged.labelM], axis=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [ ]:
dataset.insert(3, "alt", 0)

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset = dataset.astype({'user':'int64', 'lon':'float64','lat':'float64', 'alt':'float64', 'ts':'int64', 'labelM':'int64'})

In [ ]:
dataset.ts = dataset.ts/1e9

In [ ]:
np.unique(dataset['user'])

array([ 1,  2,  3,  4,  5,  6,  7,  9, 10])

In [ ]:
np.unique(dataset['labelM'])

array([0, 1, 2, 3, 4, 5])

In [ ]:
dataset.head()

,user,lon,lat,alt,ts,labelM
0,1,66.441137,29.344040,0.0,1.529925e+09,0
1,1,66.441137,29.344040,0.0,1.529925e+09,0
2,1,66.441108,29.343999,0.0,1.529925e+09,0
3,1,66.441055,29.343925,0.0,1.529925e+09,0
4,1,66.441145,29.344060,0.0,1.529925e+09,0


In [ ]:
pd.DataFrame(np.load(f'{working_path}/Geolife/numpyL/user/labelled_user_13.npy')).head(5)

,0,1,2,3,4,5
0,13.0,39.975283,116.330467,226.377953,1.192764e+09,3.0
1,13.0,39.975117,116.329083,226.377953,1.192764e+09,3.0
2,13.0,39.979433,116.327950,157.480315,1.192764e+09,3.0
3,13.0,39.981400,116.327500,157.480315,1.192764e+09,3.0
4,13.0,39.983833,116.327267,196.850394,1.192765e+09,3.0


This data preparation process is the dense and concise version described in `DataPreparation.ipynb`.

ImportError: ignored

In [ ]:

def create_datasets_RNN(data, seq_size):
    users = []
    labels = []
    user_list = np.unique(data['user'])

    #print(user_list)
    for user in user_list:
            user_data = data[data['user']==user]
            #print(user_data.shape)
            user_data_sorted = user_data.sort_values(by=['ts'])
            assert user_data_sorted['ts'].is_monotonic_increasing == True
            user_data_sorted = np.array(user_data_sorted)
            # split into trips
            trips = [] # (trip, segements, 4)
            i = 0
            for j in range(1,len(user_data_sorted)):
                if user_data_sorted[j,4] - user_data_sorted[j-1,4] <= 1200:
                    continue
                # print(np.array(user_data_sorted[i:j]))
                trips.append(user_data_sorted[i:j])
                i = j
            # print(len(trips))
            
            # split into segements for each trip
            for ind,trip in enumerate(trips):  # user -> trips -> segements
                temp_segements = []
                i = 0
                for j in range(1, len(trip)):
                    if trip[j,5] == trip[j-1,5]:
                        continue
                    temp_segements.append(trip[i:j])
                    i = j
                if(temp_segements):
                    trips[ind] = temp_segements
                else:
                    trips[ind] = [trip]
                # print("size: ",len(trips[ind]))
                # assert np.array(temp_segements).ndim == 2

            # calculate channel data
            trips_2 = []
            targets = []#np.empty((1))
            for ind,trip in enumerate(trips):
                temp_segement = []
                temp_target = []
                for ind2, segement in enumerate(trip): # segement with same mode under same trip
                    segement_4d = []

                    # drop mode 100
                    if(segement[0,5] == 100):
                        continue

                    for i in range(len(segement)-2):
                        p1 = (np.radians(segement[i,1]), np.radians(segement[i,2]))
                        p2 = (np.radians(segement[i+1,1]), np.radians(segement[i+1,2]))
                        p3 = (np.radians(segement[i+2,1]), np.radians(segement[i+2,2]))
                        dist = geodesic(p1, p2).meters
                        #print(dist)
                        t = segement[i+1,4] - segement[i,4]
                        if t == 0:
                            continue
                        speed = dist / t
                        acceleration = speed / t
                        jerk = acceleration / t

                        # Bearing Rate
                        y_1 = np.sin(p2[1] - p1[1]) * np.cos(p2[0])
                        x_1 = np.cos(p1[0])*np.sin(p2[0]) - np.sin(p1[0])*np.cos(p2[0])*np.cos(p2[1] - p1[1])
                        bearing_p1 = np.degrees(np.arctan2(y_1, x_1))

                        y_2 = np.sin(p3[1] - p2[1]) * np.cos(p3[0])
                        x_2 = np.cos(p2[0])*np.sin(p3[0]) - np.sin(p2[0])*np.cos(p3[0])*np.cos(p3[1] - p2[1])
                        bearing_p2 = np.degrees(np.arctan2(y_2, x_2))

                        bearing_rate = np.abs(bearing_p2 - bearing_p1)

                        mode = segement[i,5]
                        
                        # # bus
                        # if mode == 0 and (speed > 34 or acceleration > 2): 
                        #     continue
                        # # train
                        # if mode == 1 and (speed > 34 or acceleration > 3): 
                        #     continue
                        # # walk
                        # if mode == 2 and (speed > 7 or acceleration > 3): 
                        #     continue
                        # # car
                        # if mode == 3 and (speed > 50 or acceleration > 10): 
                        #     continue
                        # # bike
                        # if mode == 4 and (speed > 12 or acceleration > 3): 
                        #     continue
                        segement_4d.append([speed, acceleration, jerk, bearing_rate])
                        
                        if len(segement_4d) % 200 == 0:                               # from 200 -> 50
                            # print(np.array(segement_4d).shape)
                            temp_segement.append(segement_4d)
                            temp_target.append(segement[0,5])
                            segement_4d = []
                            # print(np.array(segement_4d).shape)
                        if len(temp_segement) >= seq_size:
                            # print( np.array(temp_segement).shape)
                            trips_2.append(temp_segement)
                            targets.append(temp_target)
                            temp_segement = []
                            temp_target = []
            if len(trips_2) > 0:
                users.append(trips_2)
                labels.append(targets)
                # print("size(trips)", np.array(trips_2).shape)
                # print("size(target)", np.array(targets).shape)
            else:
                print("-1")
    return users, labels
        


In [ ]:
userRNN2, labelRNN2 = create_datasets_RNN(dataset, seq_size = 2)

In [ ]:
# dump RNN dateset
pd.to_pickle(userRNN2, f'{working_path}/Geolife/user_MMM_RNN_2.pkl')
pd.to_pickle(labelRNN2, f'{working_path}/Geolife/label_MMM_RNN_2.pkl')

In [ ]:
print('user')

print(len(userRNN2))

print('label')

print(len(labelRNN2))


user
9
label
9


In [ ]:
# for i in user:
#     for j in i:
#         print(np.array(j).shape)
for i in userRNN2:
    print(np.array(i).shape)
# print(np.array(user[5]).shape)
# print(len(user[1][0]))

(12, 2, 200, 4)
(12, 2, 200, 4)
(70, 2, 200, 4)
(738, 2, 200, 4)
(5, 2, 200, 4)
(2, 2, 200, 4)
(13, 2, 200, 4)
(399, 2, 200, 4)
(30, 2, 200, 4)


## Define a Recurrent Neural Network seq_8 + BiGRU

> Indented block



In [ ]:
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

#from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

Build the network based on the <span class="mark">yellow colored parameters</span>

![image.png](attachment:image.png)

In [ ]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")


Running GPU.


### define train, validation, test set

In [ ]:
import os
working_path = '/content/drive/My Drive/s182190/data'
os.chdir(f'{working_path}')

import os
import numpy as np
import pandas as pd


In [ ]:

userRNN2 = pd.read_pickle(f'{working_path}/Geolife/user_MMM_RNN_2.pkl')
labelRNN2 = pd.read_pickle(f'{working_path}/Geolife/label_MMM_RNN_2.pkl')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# split to train, valid, test set

x_remain_ind, x_test_ind, y_remain_ind, y_test_ind = train_test_split(list(range(len(userRNN2))),list(range(len(labelRNN2))),test_size=0.2, shuffle=True)
x_train_ind, x_valid_ind, y_train_ind, y_valid_ind = train_test_split(x_remain_ind,y_remain_ind,test_size =0.375, shuffle=True)



print(x_train_ind)
print(x_valid_ind)
print(x_test_ind)
print(y_train_ind)
print(y_valid_ind)
print(y_test_ind)

print(f'We have {len(x_train_ind)} users in the training set')
print(f'We have {len(x_valid_ind)} users in the validation set.')

def flatten_user_channel(x):
    return [[[c for p in sege for c in p] for sege in trip] for user in x for trip in user]
def flatten_user(x):
    return [trip for user in x for trip in user]

print(f'We have {len(x_test_ind)} users in the test set.')
x_train =np.array(flatten_user_channel([userRNN2[i] for i in x_train_ind]))
x_valid = np.array(flatten_user_channel([userRNN2[i] for i in x_valid_ind]))
x_test = np.array(flatten_user_channel([userRNN2[i] for i in x_test_ind]))

y_train = np.array(flatten_user([labelRNN2[i] for i in y_train_ind]))
y_valid = np.array(flatten_user([labelRNN2[i] for i in y_valid_ind]))
y_test = np.array(flatten_user([labelRNN2[i] for i in y_test_ind]))

print("Total segments of x_train",sum([np.array(i).shape[0] for i in x_train]), "*",  np.array(x_train[0]).shape)
print("Total segments of y_train",sum([np.array(i).shape[0] for i in y_train]), "*", np.array(y_train[0]).shape)

print("Total segments of x_valid",sum([np.array(i).shape[0] for i in x_valid]), "*",  np.array(x_valid[0]).shape)
print("Total segments of y_valid",sum([np.array(i).shape[0] for i in y_valid]),"*", np.array(y_valid[0]).shape)

print("Total segments of x_test",sum([np.array(i).shape[0] for i in x_test]), "*",  np.array(x_test[0]).shape)
print("Total segments of y_test",sum([np.array(i).shape[0] for i in y_test]), "*", np.array(y_test[0]).shape)

[7, 5, 0, 2]
[3, 1, 4]
[6, 8]
[7, 5, 0, 2]
[3, 1, 4]
[6, 8]
We have 4 users in the training set
We have 3 users in the validation set.
We have 2 users in the test set.
Total segments of x_train 966 * (2, 800)
Total segments of y_train 966 * (2,)
Total segments of x_valid 1510 * (2, 800)
Total segments of y_valid 1510 * (2,)
Total segments of x_test 86 * (2, 800)
Total segments of y_test 86 * (2,)


In [ ]:
# Define a loss function and optimizer for this problem
y_train_mode_spread = [sege for trip in y_train for sege in trip]
print(y_train_mode_spread)
unique, counts = np.unique(y_train_mode_spread, return_counts=True)
print(unique, counts)
weights = torch.FloatTensor(1/counts * np.min(counts))
print(weights)

# Define a loss function and optimizer for this problem
y_val_mode_spread = [sege for trip in y_valid for sege in trip]
print(y_val_mode_spread)
unique, counts = np.unique(y_val_mode_spread, return_counts=True)
print(unique, counts)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

1. Batch size / shuffle
2. LSTM / GRU
3. Confusion Matrix



In [ ]:
np.array(x_train).shape

(483, 2, 800)

### Define Network

In [ ]:
# Hyper-parameters
input_size = 800
hidden_size = 5

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.rnn = nn.GRU(input_size, hidden_size = hidden_size, num_layers = 4, bidirectional = True, bias = False, batch_first = True ) 
        #discuss bidirectional and compare with Gru and LSTM
        self.dropout = nn.Dropout2d(p = 0.5)
        self.layernorm = nn.LayerNorm([2* hidden_size])
        self.l_out1 = nn.Linear(in_features=2*hidden_size,
                            out_features= 10,
                            bias=False)
        self.l_out2 = nn.Linear(in_features=10,
                            out_features= 5,
                            bias=False)
        

    def forward(self, x):

        # RNN returns output and last hidden state
        # print("1    ", x.shape)
        x, h = self.rnn(x)

        x = self.layernorm(x)
        # print("2    ", x.shape)
        # Flatten output for feed-forward layer
        x = relu(x.view(-1, 2 * self.rnn.hidden_size))
        # print("3    ", x.shape)
        # x = self.dropout(x)
        # Output layer
        # x = self.l_out1(x)
        x = relu(self.l_out1(x))
        # print("4    ", x.shape)
        x = relu(self.l_out2(x))

        
        return x

In [ ]:
# Initialize a new network
net = Net()
if use_cuda:
    net.cuda()
print(net)


criterion = nn.CrossEntropyLoss(weight= weights.cuda())
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr =0.005, betas=(0.9, 0.999),eps = 1e-08, weight_decay=0.0001 ) # early stop/ auto decay
#optimizer = torch.optim.SGD(net.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0001 ) # early stop/ auto decay

#ADAM lr=0.001, wc=0.0001


Net(
  (rnn): GRU(800, 5, num_layers=4, bias=False, batch_first=True, bidirectional=True)
  (dropout): Dropout2d(p=0.5, inplace=False)
  (layernorm): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  (l_out1): Linear(in_features=10, out_features=10, bias=False)
  (l_out2): Linear(in_features=10, out_features=5, bias=False)
)


In [ ]:
np.sum([np.product(x.detach().cpu().numpy().shape) for x in net.parameters()])

25660

In [ ]:
np.array(x_train[4]).shape

(2, 800)

### Train the network

In [ ]:
batch_size = 10
num_epochs = 6
num_samples_train = x_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = x_valid.shape[0]
num_batches_valid = num_samples_valid // batch_size

train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
training_loss = []
validation_loss = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    cur_loss = 0
    # Track loss
    epoch_training_loss = 0
    epoch_validation_loss = 0
    net.train()
    for i in range(num_batches_train):
        slc = get_slice(i, batch_size)
        x_batch = Variable(torch.from_numpy(x_train[slc]))
        # print(x_batch.shape)
        output = net(x_batch.float().cuda())
        
        # compute gradients given loss
        y_batch = Variable(torch.from_numpy(y_train[slc]).long()).reshape(-1)
        # print(output.shape, y_batch.shape)
        loss = criterion(output, y_batch.cuda())
        optimizer.zero_grad()

        # for item in net.named_parameters():
        #     h = item[1].register_hook(lambda grad: print(grad))
        loss.backward()

        # prevent gradient exploding
        torch.nn.utils.clip_grad_norm(net.parameters(), 0.25)
        optimizer.step()
        
        # Update loss
        epoch_training_loss += loss.detach().cpu().numpy()

    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    with torch.no_grad(): # 禁用梯度计算
        for i in range(num_batches_train):
            slc = get_slice(i, batch_size)
            # print(slc)
            x_batch = Variable(torch.from_numpy(x_train[slc]))
            
            output = net(x_batch.float().cuda()) # [seq, batch, num_direction*hidden_size]
            # print(output.shape)
            preds = torch.max(output, 1)[1]
            # print(preds.shape)
            # print('preds:', preds)
            
            train_targs += list(y_train[slc].reshape(-1))
            train_preds += list(preds.cpu().data.numpy())

    
    ### Evaluate validation
    val_preds, val_targs = [], []
    with torch.no_grad(): # 禁用梯度计算
        for i in range(num_batches_valid):
            slc = get_slice(i, batch_size)
            x_batch = Variable(torch.from_numpy(x_valid[slc]))
            # print(x_batch)
            output = net(x_batch.float().cuda())
            # print(output)
            preds = torch.max(output, 1)[1]
            # print('preds:', preds)
            
            val_preds += list(preds.cpu().data.numpy())
            val_targs += list(y_valid[slc].reshape(-1))
            # Compute loss
            loss = criterion(output, y_batch.cuda())
            # Update loss
            epoch_validation_loss += loss.detach().cpu().numpy()

    train_acc_cur = accuracy_score(train_targs, train_preds)

    # print(val_targs)
    # print(val_preds)
    valid_acc_cur = accuracy_score(val_targs, val_preds)
    
    train_acc.append(train_acc_cur)
    valid_acc.append(valid_acc_cur)
    
    # Save loss for plot
    training_loss.append(epoch_training_loss/len(x_train))
    validation_loss.append(epoch_validation_loss/len(x_valid))
    
    if epoch % 10 == 0:
        print("Epoch %2i : Train Loss %f , Validation Loss %f, Train acc %f, Valid acc %f" % (
                epoch+1, training_loss[-1], validation_loss[-1], train_acc_cur, valid_acc_cur))
        
    if epoch == num_epochs-1:
        print(val_targs, val_preds)
        cm = confusion_matrix(val_targs, val_preds, labels = [0, 1, 2, 3, 4])
        ConfusionMatrixDisplay(cm, display_labels= ['BUS','Train & Subway', 'Walk & Run', 'Car & Taxi & Motocycle', 'Bike?']).plot(values_format = 'd')
        print(classification_report(val_targs, val_preds, target_names = ['BUS','Train & Subway', 'Walk & Run', 'Car & Taxi & Motocycle', 'Bike?']))


# Plot training and validation loss
import matplotlib.pyplot as plt
epoch = np.arange(len(training_loss))
plt.figure()
plt.plot(epoch, training_loss, 'r', label='Training loss',)
plt.plot(epoch, validation_loss, 'b', label='Validation loss')
plt.legend()
plt.xlabel('Epoch'), plt.ylabel('NLL')
plt.show()


RuntimeError: ignored

In [ ]:
import matplotlib.pyplot as plt

epoch = np.arange(len(train_acc))
plt.figure()
print(train_acc)
print(valid_acc)
plt.plot(epoch, train_acc, 'r', epoch, valid_acc, 'b')
plt.legend(['Train Acc', 'Val Acc'])
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.show()

### Evaluate test set
x_batch = Variable(torch.from_numpy(x_test))
output = net(x_batch.float().cuda())
preds = torch.max(output, 1)[1]
print("\nTest set Acc:  %f" % (accuracy_score(list(y_test.reshape(-1)), list(preds.cpu().data.numpy()))))

In [ ]:
len(x[4][3])

# Data Preparation, MMM Data Way

### Load Data

#### Load GPS+GIS fusion tensors - dimensionality 9x9x11 - ( [see explaination here](https://www.sciencedirect.com/science/article/pii/S0968090X20307385?dgcid=author#bfn4))

Remember to delete "image_data" after training, before loading "image_data" with the user_test partition, so that the dataloader can perform the test on your network, with weights and biases resulting from the training.

In [ ]:
import os
working_path = '/content/drive/My Drive/s182190/data'
os.chdir(f'{working_path}')
from src.data_utils_conda import load_user_data


In [ ]:
def create_data_frame(user, ts, pos_webm, dummy_array, labels2, labelsM, labelsP, segmentation = False, seq_cutoff_time = 300, seq_cutoff_speed = 42, Standardize = False):
    """
    """
    df = pd.DataFrame({
        'user': user,
        'ts': ts,
        'image_ix': np.arange(0, ts.shape[0]),
        'x': pos_webm[:,0],
        'y': pos_webm[:,1],
        'x_web': pos_webm[:,0],
        'y_web': pos_webm[:,1],
        'label2': labels2, 
        'labelP': labelsP, 
        'labelM': labelsM
    })
    
    if dummy_array is not None:
        
        df['f_highway_motorway'] = dummy_array[:,0]
        df['f_traffic_signals'] = dummy_array[:,1]
        df['f_bus_stops'] = dummy_array[:,2]
        df['f_landuse_meadow'] = dummy_array[:,3]
        df['f_landuse_residential'] = dummy_array[:,4]
        df['f_landuse_industrial'] = dummy_array[:,5]
        df['f_landuse_commercial'] = dummy_array[:,6]
        df['f_shop'] = dummy_array[:,7]
        df['f_railways'] = dummy_array[:,8]
        df['f_railways_station'] = dummy_array[:,9]
        df['f_subway'] = dummy_array[:,10]
        
    
    df.sort_values('ts', inplace = True)
    
    df['delta_t'] = np.concatenate([[0], (df['ts'].values[1:] - df['ts'].values[:-1]) / pd.to_timedelta('1s')], axis = 0)    
    df['delta_d'] = np.concatenate([[0], np.linalg.norm(df[['x', 'y']].values[1:] - df[['x', 'y']].values[:-1], axis = 1)], axis = 0)    
    df['bearing'] = np.concatenate([[0], np.arctan2(df[['y']].values[1:] - df[['y']].values[:-1], df[['x']].values[1:] - df[['x']].values[:-1]).reshape(-1)], axis = 0)
    df['speed'] = df['delta_d'] / df['delta_t']
    df['acceleration'] = df['speed']/df['delta_t']
    df['jerk'] = df['acceleration']/ df['delta_t']
    
    cut_labels_6 = [0, 1, 2, 3, 4]
    cut_bins = [0, 6, 10, 14, 18, 24]
    df['tod'] = pd.cut(df['ts'].dt.hour, bins=cut_bins, labels=cut_labels_6, right = False).astype(int)
    
    df = df[lambda x: x['delta_t'] > 0].copy()
    
    seq_bins = np.cumsum((df['delta_t'] >= seq_cutoff_time) | (df['speed'] > seq_cutoff_speed))
    seq_bin_ids, seq_bin_counts = np.unique(seq_bins, return_counts=True)
    
    if segmentation:
        df['segment_id'] = seq_bins
        df['segment_ix'] = np.concatenate([np.arange(seq_bin_counts[i]) for i in range(len(seq_bin_counts))])
        df['segment_point_count'] = np.repeat(seq_bin_counts, seq_bin_counts)
    
    if Standardize:
        df = scale_by_ID(df, ['delta_d','bearing','speed'])
    
    return df

In [ ]:
seq_cutoff_speed = 45
seq_cutoff_time = 60
filter_seq = 5

dataset = pd.concat([create_data_frame(*load_user_data(user, data_directory=f'{working_path}/interm/Pickle'), segmentation=True, seq_cutoff_time = seq_cutoff_time, seq_cutoff_speed = seq_cutoff_speed) for user in range(1,13)]).reset_index(drop=True)
dataset.head(10)

,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count
0,1,2018-06-25 09:57:31+00:00,1,1.394209e+06,7.516646e+06,1.394209e+06,7.516646e+06,1,3,0,671.0,7.751420,-1.856730,0.011552,0.000017,2.565746e-08,1,1,0,1
1,1,2018-06-25 09:58:43+00:00,4,1.394159e+06,7.516558e+06,1.394159e+06,7.516558e+06,2,0,4,72.0,133.415960,-2.005347,1.852999,0.025736,3.574459e-04,1,2,0,1
2,1,2018-06-25 09:58:44+00:00,5,1.394177e+06,7.516504e+06,1.394177e+06,7.516504e+06,2,0,4,1.0,57.104376,-1.252389,57.104376,57.104376,5.710438e+01,1,3,0,1
3,1,2018-06-25 09:58:45+00:00,6,1.394163e+06,7.516550e+06,1.394163e+06,7.516550e+06,2,0,4,1.0,47.314571,1.864483,47.314571,47.314571,4.731457e+01,1,4,0,606
4,1,2018-06-25 09:58:46+00:00,7,1.394156e+06,7.516552e+06,1.394156e+06,7.516552e+06,2,0,4,1.0,7.512082,2.876788,7.512082,7.512082,7.512082e+00,1,4,1,606
5,1,2018-06-25 09:58:47+00:00,8,1.394154e+06,7.516552e+06,1.394154e+06,7.516552e+06,2,0,4,1.0,2.106983,2.983060,2.106983,2.106983,2.106983e+00,1,4,2,606
6,1,2018-06-25 09:58:48+00:00,9,1.394152e+06,7.516548e+06,1.394152e+06,7.516548e+06,2,0,4,1.0,3.969750,-2.004935,3.969750,3.969750,3.969750e+00,1,4,3,606
7,1,2018-06-25 09:58:49+00:00,10,1.394151e+06,7.516547e+06,1.394151e+06,7.516547e+06,2,0,4,1.0,1.838343,-2.005848,1.838343,1.838343,1.838343e+00,1,4,4,606
8,1,2018-06-25 09:58:50+00:00,11,1.394151e+06,7.516545e+06,1.394151e+06,7.516545e+06,2,0,4,1.0,1.299092,-2.088964,1.299092,1.299092,1.299092e+00,1,4,5,606
9,1,2018-06-25 09:58:51+00:00,12,1.394149e+06,7.516544e+06,1.394149e+06,7.516544e+06,2,0,4,1.0,2.121588,-2.376706,2.121588,2.121588,2.121588e+00,1,4,6,606


### add mode and purpose description column

In [ ]:
labels = pd.read_csv(f'{working_path}/raw/UID-MMM-labels.csv').drop(columns='Unnamed: 0')[['stop','mode','howTravelled','purpose','mainActivity']]
_, unique_mode_index = np.unique(labels['mode'], return_index = True)
_, unique_purpose_index = np.unique(labels['purpose'], return_index = True)
mode_label = pd.DataFrame(labels, index= unique_mode_index, columns= ['mode', 'howTravelled']).set_index('mode', inplace= False)
print(mode_label)

# the AIR mode has some issue, the default mode is a chaos
purpose_label = pd.DataFrame(labels, index= unique_purpose_index, columns= ['purpose', 'mainActivity']).set_index('purpose', inplace= False)
print(purpose_label)

# add howtravelled and activity
dataset['mode'] = dataset.apply(lambda row: mode_label.loc[row.labelM], axis=1)
dataset['purpose'] = dataset.apply(lambda row: purpose_label.loc[row.labelP], axis=1)

     howTravelled
mode             
0            -100
1             Air
2         Bicycle
3             Bus
4         Default
5            Foot
6      Light Rail
7           Other
8         Vehicle
          mainActivity
purpose               
0                 -100
1            Accompany
2          Change Mode
3              Eat Out
4            Education
5        Entertainment
6              Errands
7             Exercise
8                 Home
9              Medical
10               Other
11         Second Home
12            Shopping
13           Socialize
14                Work
15        Work Related


In [ ]:
dataset.head()

,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count,mode,purpose
0,1,2018-06-25 09:57:31+00:00,1,1.394209e+06,7.516646e+06,1.394209e+06,7.516646e+06,1,3,0,671.0,7.751420,-1.856730,0.011552,0.000017,2.565746e-08,1,1,0,1,-100,Eat Out
1,1,2018-06-25 09:58:43+00:00,4,1.394159e+06,7.516558e+06,1.394159e+06,7.516558e+06,2,0,4,72.0,133.415960,-2.005347,1.852999,0.025736,3.574459e-04,1,2,0,1,Default,-100
2,1,2018-06-25 09:58:44+00:00,5,1.394177e+06,7.516504e+06,1.394177e+06,7.516504e+06,2,0,4,1.0,57.104376,-1.252389,57.104376,57.104376,5.710438e+01,1,3,0,1,Default,-100
3,1,2018-06-25 09:58:45+00:00,6,1.394163e+06,7.516550e+06,1.394163e+06,7.516550e+06,2,0,4,1.0,47.314571,1.864483,47.314571,47.314571,4.731457e+01,1,4,0,606,Default,-100
4,1,2018-06-25 09:58:46+00:00,7,1.394156e+06,7.516552e+06,1.394156e+06,7.516552e+06,2,0,4,1.0,7.512082,2.876788,7.512082,7.512082,7.512082e+00,1,4,1,606,Default,-100


In [ ]:
pd.to_pickle(dataset,f"{working_path}/Geolife/dataset_temp.pkl")

In [ ]:
dataset = pd.read_pickle(f"{working_path}/Geolife/dataset_temp.pkl")

In [ ]:
dataset.ts[0]

Timestamp('2018-06-25 09:57:31+0000', tz='UTC')

In [ ]:
# filter by hour
dataset['str_ts']= dataset.ts.dt.strftime('%Y-%m-%d-%H')
filter_list_hour = []
for i in dataset.user.unique():
    hour_list = dataset[(dataset.user == i) & (dataset.labelM == 4)].str_ts.unique()
    filter_list_hour.append(dataset[(dataset.user == i) & (~dataset.str_ts.isin(hour_list))])
                                  
dataset_filter_by_hour = pd.concat(filter_list_hour, ignore_index=True)
dataset_filter_by_hour.to_pickle(f'{working_path}/Geolife/dataset_filter_by_hour.pkl')
dataset_filter_by_hour.user.unique()


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [ ]:
dataset.head(10)

,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count,mode,purpose,str_ts
0,1,2018-06-25 09:57:31+00:00,1,1.394209e+06,7.516646e+06,1.394209e+06,7.516646e+06,1,3,0,671.0,7.751420,-1.856730,0.011552,0.000017,2.565746e-08,1,1,0,1,-100,Eat Out,2018-06-25-09
1,1,2018-06-25 09:58:43+00:00,4,1.394159e+06,7.516558e+06,1.394159e+06,7.516558e+06,2,0,4,72.0,133.415960,-2.005347,1.852999,0.025736,3.574459e-04,1,2,0,1,Default,-100,2018-06-25-09
2,1,2018-06-25 09:58:44+00:00,5,1.394177e+06,7.516504e+06,1.394177e+06,7.516504e+06,2,0,4,1.0,57.104376,-1.252389,57.104376,57.104376,5.710438e+01,1,3,0,1,Default,-100,2018-06-25-09
3,1,2018-06-25 09:58:45+00:00,6,1.394163e+06,7.516550e+06,1.394163e+06,7.516550e+06,2,0,4,1.0,47.314571,1.864483,47.314571,47.314571,4.731457e+01,1,4,0,606,Default,-100,2018-06-25-09
4,1,2018-06-25 09:58:46+00:00,7,1.394156e+06,7.516552e+06,1.394156e+06,7.516552e+06,2,0,4,1.0,7.512082,2.876788,7.512082,7.512082,7.512082e+00,1,4,1,606,Default,-100,2018-06-25-09
5,1,2018-06-25 09:58:47+00:00,8,1.394154e+06,7.516552e+06,1.394154e+06,7.516552e+06,2,0,4,1.0,2.106983,2.983060,2.106983,2.106983,2.106983e+00,1,4,2,606,Default,-100,2018-06-25-09
6,1,2018-06-25 09:58:48+00:00,9,1.394152e+06,7.516548e+06,1.394152e+06,7.516548e+06,2,0,4,1.0,3.969750,-2.004935,3.969750,3.969750,3.969750e+00,1,4,3,606,Default,-100,2018-06-25-09
7,1,2018-06-25 09:58:49+00:00,10,1.394151e+06,7.516547e+06,1.394151e+06,7.516547e+06,2,0,4,1.0,1.838343,-2.005848,1.838343,1.838343,1.838343e+00,1,4,4,606,Default,-100,2018-06-25-09
8,1,2018-06-25 09:58:50+00:00,11,1.394151e+06,7.516545e+06,1.394151e+06,7.516545e+06,2,0,4,1.0,1.299092,-2.088964,1.299092,1.299092,1.299092e+00,1,4,5,606,Default,-100,2018-06-25-09
9,1,2018-06-25 09:58:51+00:00,12,1.394149e+06,7.516544e+06,1.394149e+06,7.516544e+06,2,0,4,1.0,2.121588,-2.376706,2.121588,2.121588,2.121588e+00,1,4,6,606,Default,-100,2018-06-25-09


In [ ]:
# Drop the records with both -100 on mode and purpose
dataset_filter_by_hour = pd.read_pickle(f'{working_path}/Geolife/dataset_filter_by_hour.pkl')

dataset_filter_by_hour_drop = dataset_filter_by_hour[(dataset_filter_by_hour.labelM != 0) | (dataset_filter_by_hour.labelP != 0)]
print(dataset_filter_by_hour_drop[['label2', 'labelM', 'mode', 'labelP','purpose']].value_counts(sort = False))

dataset_filter_by_hour_drop.to_pickle(f'{working_path}/Geolife/dataset_filter_by_hour_drop.pkl')

label2  labelM  mode        labelP  purpose      
1       0       -100        1       Accompany          7102
                            2       Change Mode        7177
                            3       Eat Out            3316
                            4       Education          9761
                            5       Entertainment        87
                            6       Errands              57
                            7       Exercise            224
                            8       Home             535611
                            9       Medical            1296
                            11      Second Home         105
                            12      Shopping            504
                            13      Socialize          4365
                            14      Work             131974
                            15      Work Related        114
2       1       Air         0       -100              51456
        2       Bicycle     0       -100          

### Merge classes

In [ ]:
purpose_names = ["-100", "Accompany", "Change Mode", "Informal", "Formal","Home"]
mode_names = ["-100", "Mix","Bike","Bus","Walk","Light Rail"]

In [ ]:
dataset = pd.read_pickle(f'{working_path}/Geolife/dataset_filter_by_hour_drop.pkl')
dataset[['label2', 'labelM', 'mode', 'labelP','purpose']].value_counts(sort = False)

label2  labelM  mode        labelP  purpose      
1       0       -100        1       Accompany          7102
                            2       Change Mode        7177
                            3       Eat Out            3316
                            4       Education          9761
                            5       Entertainment        87
                            6       Errands              57
                            7       Exercise            224
                            8       Home             535611
                            9       Medical            1296
                            11      Second Home         105
                            12      Shopping            504
                            13      Socialize          4365
                            14      Work             131974
                            15      Work Related        114
2       1       Air         0       -100              51456
        2       Bicycle     0       -100          

In [ ]:

# merge classes
dict_labelM = {1:1, 2:2, 3:3, 5:4, 6:5, 7:1, 8:1}
dict_mode = {"Air":"Mix", "Bicycle":"Bike","Bus":"Bus","Foot":"Walk","Light Rail":"Light Rail","Other":"Mix","Vehicle":"Mix"}

dict_labelP = {1:1, 2:2, 3:3, 4:4, 5:3, 6:4, 7:3, 8:5,9:3, 11:5, 12:3,13:3, 14:4, 15:4}
dict_purpose = {"Accompany":"Accompany" , "Change Mode":"Change Mode","Eat Out": "Informal", "Education": "Formal", "Entertainment":"Informal","Errands":"Formal","Exercise":"Informal","Home":"Home","Medical":"Informal","Second Home":"Home", "Shopping":"Informal","Socialize":"Informal","Work":"Formal","Work Related":"Formal"}


In [ ]:

dataset['labelM'] = dataset['labelM'].map(dict_labelM).fillna(dataset['labelM'])
dataset['mode'] = dataset['mode'].map(dict_mode).fillna(dataset['mode'])
dataset['labelP'] = dataset['labelP'].map(dict_labelP).fillna(dataset['labelP'])
dataset['purpose'] = dataset['purpose'].map(dict_purpose).fillna(dataset['purpose'])

# drop user
# dataset = dataset[dataset.user == 2]
# dataset.head()
print(dataset[['label2', 'labelM', 'mode', 'labelP','purpose']].value_counts(sort = False))

dataset.to_pickle(f'{working_path}/Geolife/dataset_merged.pkl')

label2  labelM  mode        labelP  purpose    
1       0.0     -100        1.0     Accompany        7102
                            2.0     Change Mode      7177
                            3.0     Informal         9792
                            4.0     Formal         141906
                            5.0     Home           535716
2       1.0     Mix         0.0     -100            54235
        2.0     Bike        0.0     -100             3893
        3.0     Bus         0.0     -100            72336
        4.0     Walk        0.0     -100             1254
        5.0     Light Rail  0.0     -100            14680
dtype: int64


In [ ]:
dataset.head()

,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count,mode,purpose,str_ts
0,1,2018-06-25 11:05:34+00:00,1483,1.394210e+06,7.516647e+06,1.394210e+06,7.516647e+06,1,3.0,0.0,1775.0,7.265090,-1.759773,0.004093,0.000002,1.299110e-09,2,10,0,6,-100,Informal,2018-06-25-11
1,1,2018-06-25 11:05:35+00:00,1484,1.394210e+06,7.516647e+06,1.394210e+06,7.516647e+06,1,3.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2,10,1,6,-100,Informal,2018-06-25-11
2,1,2018-06-25 11:06:16+00:00,1487,1.394210e+06,7.516643e+06,1.394210e+06,7.516643e+06,1,3.0,0.0,41.0,4.633838,-1.749762,0.113020,0.002757,6.723405e-05,2,10,2,6,-100,Informal,2018-06-25-11
3,1,2018-06-25 11:07:07+00:00,1488,1.394208e+06,7.516636e+06,1.394208e+06,7.516636e+06,1,3.0,0.0,51.0,6.745727,-1.754004,0.132269,0.002594,5.085319e-05,2,10,3,6,-100,Informal,2018-06-25-11
4,1,2018-06-25 11:07:41+00:00,1491,1.394211e+06,7.516648e+06,1.394211e+06,7.516648e+06,1,3.0,0.0,34.0,10.506570,1.359983,0.309017,0.009089,2.673155e-04,2,10,4,6,-100,Informal,2018-06-25-11


### Create train validation and test partitions

Since user 4 and 9 have lots of points, We use k parameter to define a specific slice of the train, validation and test collection. If you want to learn more, please go to the data utilis.py. 

For the formal test later of the algorithm, the -random- parameter is set to be True, k is not invoked.

In [ ]:
dataset = pd.read_pickle(f'{working_path}/Geolife/dataset_merged.pkl')

In [ ]:
dataset.head()

,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count,mode,purpose,str_ts
0,1,2018-06-25 11:05:34+00:00,1483,1.394210e+06,7.516647e+06,1.394210e+06,7.516647e+06,1,3.0,0.0,1775.0,7.265090,-1.759773,0.004093,0.000002,1.299110e-09,2,10,0,6,-100,Informal,2018-06-25-11
1,1,2018-06-25 11:05:35+00:00,1484,1.394210e+06,7.516647e+06,1.394210e+06,7.516647e+06,1,3.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2,10,1,6,-100,Informal,2018-06-25-11
2,1,2018-06-25 11:06:16+00:00,1487,1.394210e+06,7.516643e+06,1.394210e+06,7.516643e+06,1,3.0,0.0,41.0,4.633838,-1.749762,0.113020,0.002757,6.723405e-05,2,10,2,6,-100,Informal,2018-06-25-11
3,1,2018-06-25 11:07:07+00:00,1488,1.394208e+06,7.516636e+06,1.394208e+06,7.516636e+06,1,3.0,0.0,51.0,6.745727,-1.754004,0.132269,0.002594,5.085319e-05,2,10,3,6,-100,Informal,2018-06-25-11
4,1,2018-06-25 11:07:41+00:00,1491,1.394211e+06,7.516648e+06,1.394211e+06,7.516648e+06,1,3.0,0.0,34.0,10.506570,1.359983,0.309017,0.009089,2.673155e-04,2,10,4,6,-100,Informal,2018-06-25-11


In [ ]:
user_train, user_val, user_test = train, val, test = train_test_data_split(dataset.user.unique(),Random=False, k=9) # use specified splits, check the source code
print(f'partition: train {user_train}, validation {user_val}, test {user_test}')

data_train =  pd.concat([dataset[(dataset.user==tr)] for tr in user_train]).reset_index(drop=True)
data_val = pd.concat([dataset[(dataset.user==va)] for va in user_val]).reset_index(drop=True)
data_test = pd.concat([dataset[(dataset.user==te)] for te in user_test]).reset_index(drop=True)

print(f'length: train {len(data_train)}, validation {len(data_val)}, test {len(data_test)}')

data_train.head(10)

partition: train [4, 1, 8, 12, 11, 7, 10, 5], validation [9], test [3, 2, 6]
length: train 585391, validation 200669, test 62031


,user,ts,image_ix,x,y,x_web,y_web,label2,labelP,labelM,delta_t,delta_d,bearing,speed,acceleration,jerk,tod,segment_id,segment_ix,segment_point_count,mode,purpose,str_ts
0,4,2018-06-15 05:59:34+00:00,3,1.405763e+06,7.485806e+06,1.405763e+06,7.485806e+06,2,0.0,5.0,126.0,167.688110,-1.169201,1.330858,0.010562,8.382830e-05,0,1,0,1,Light Rail,-100,2018-06-15-05
1,4,2018-06-15 08:03:34+00:00,3684,1.394213e+06,7.516659e+06,1.394213e+06,7.516659e+06,1,4.0,0.0,2139.0,6.735206,1.248921,0.003149,0.000001,6.882061e-10,1,29,0,2,-100,Formal,2018-06-15-08
2,4,2018-06-15 08:03:51+00:00,3685,1.394209e+06,7.516648e+06,1.394209e+06,7.516648e+06,1,4.0,0.0,17.0,10.977184,-1.906872,0.645717,0.037983,2.234314e-03,1,29,1,2,-100,Formal,2018-06-15-08
3,4,2018-06-15 08:05:13+00:00,3688,1.394214e+06,7.516664e+06,1.394214e+06,7.516664e+06,1,4.0,0.0,82.0,16.436425,1.248127,0.200444,0.002444,2.981026e-05,1,30,0,2,-100,Formal,2018-06-15-08
4,4,2018-06-15 08:05:54+00:00,3689,1.394213e+06,7.516659e+06,1.394213e+06,7.516659e+06,1,4.0,0.0,41.0,5.115697,-1.864637,0.124773,0.003043,7.422552e-05,1,30,1,2,-100,Formal,2018-06-15-08
5,4,2018-06-15 08:07:37+00:00,3692,1.394216e+06,7.516667e+06,1.394216e+06,7.516667e+06,1,4.0,0.0,103.0,8.163831,1.237253,0.079260,0.000770,7.471062e-06,1,31,0,1,-100,Formal,2018-06-15-08
6,4,2018-06-15 08:20:13+00:00,3693,1.394215e+06,7.516665e+06,1.394215e+06,7.516665e+06,1,4.0,0.0,756.0,2.301488,-1.928052,0.003044,0.000004,5.326518e-09,1,32,0,1,-100,Formal,2018-06-15-08
7,4,2018-06-15 08:22:27+00:00,3694,1.394213e+06,7.516659e+06,1.394213e+06,7.516659e+06,1,4.0,0.0,134.0,6.354567,-1.932308,0.047422,0.000354,2.641019e-06,1,33,0,1,-100,Formal,2018-06-15-08
8,4,2018-06-15 08:24:21+00:00,3696,1.394216e+06,7.516667e+06,1.394216e+06,7.516667e+06,1,4.0,0.0,114.0,8.325730,1.206465,0.073033,0.000641,5.619631e-06,1,34,0,1,-100,Formal,2018-06-15-08
9,4,2018-06-15 08:26:41+00:00,3697,1.394216e+06,7.516667e+06,1.394216e+06,7.516667e+06,1,4.0,0.0,140.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1,35,0,1,-100,Formal,2018-06-15-08


#### Define Image Tensor Dataloader

We implement our own Tensor Dataset in order to be able to do fast lookup of sequences and images. Just add features inside
```
df[['feat1','feat2',...]]
```

In [ ]:
import torch
class ImageTensorDataset(torch.utils.data.Dataset):
    filter_seq = 7
    def __init__(self, df,filter_seq=filter_seq):
        self.seq = np.stack([np.roll(df[['speed', 'acceleration','jerk','bearing']].values, i, axis = 0) for i in range(filter_seq, -1, -1)], axis = 1)
        self.seq = self.seq[df['segment_ix'] >= filter_seq]

        self.labelM = df[df['segment_ix'] >= filter_seq]['labelM'].values
     
        self.user_id = df[df['segment_ix'] >= filter_seq]['user'].values

        
    def __len__(self):
        return len(self.labelM)
    
    def __getitem__(self, key):
        return self.seq[key],  self.labelM[key]

demo_dataset = ImageTensorDataset(data_train)
demo_loader = torch.utils.data.DataLoader(demo_dataset, batch_size=3, shuffle=True)

for X_seq, y in demo_loader:

    print('X_seq :', X_seq.shape)
    print(X_seq[0, :])

    print('y:', y.shape)
    print(y[0])

    break;

X_seq : torch.Size([3, 8, 4])
tensor([[ 0.5238,  0.5238,  0.5238,  1.7936],
        [ 1.0097,  1.0097,  1.0097,  0.9289],
        [ 1.3630,  1.3630,  1.3630, -1.2800],
        [ 0.4924,  0.4924,  0.4924, -0.8528],
        [ 1.1999,  1.1999,  1.1999, -1.1031],
        [ 0.9478,  0.9478,  0.9478, -1.6496],
        [ 0.2222,  0.2222,  0.2222, -1.6762],
        [ 0.1543,  0.1543,  0.1543, -1.2402]], dtype=torch.float64)
y: torch.Size([3])
tensor(0., dtype=torch.float64)


In [ ]:
len(demo_loader)

181774

#### define train, val, test set and dataloader

In [ ]:
    train_dataset = ImageTensorDataset(data_train)
    val_dataset = ImageTensorDataset(data_val)
    test_dataset = ImageTensorDataset(data_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=13500, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=13500, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=13500, shuffle=True)


## Define a Recurrent Neural Network

In [ ]:
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

#from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout2d
#from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

Build the network based on the <span class="mark">yellow colored parameters</span>

![image.png](attachment:image.png)

In [ ]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")


def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

No GPU available.


In [ ]:
input_size = 200
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.rnn = nn.GRU(input_size, hidden_size = hidden_size, num_layers = 4, bidirectional = True, bias = False, batch_first = True ) 
        #discuss bidirectional and compare with Gru and LSTM
        self.dropout = nn.Dropout2d(p = 0.5)
        self.layernorm = nn.LayerNorm([2* hidden_size])
        self.l_out1 = nn.Linear(in_features=2*hidden_size,
                            out_features= 10,
                            bias=False)
        self.l_out2 = nn.Linear(in_features=10,
                            out_features= 5,
                            bias=False)
        

    def forward(self, x):

        # RNN returns output and last hidden state
        # print("1    ", x.shape)
        x, h = self.rnn(x)

        x = self.layernorm(x)
        # print("2    ", x.shape)
        # Flatten output for feed-forward layer
        x = relu(x.view(-1, 2 * self.rnn.hidden_size))
        # print("3    ", x.shape)
        # x = self.dropout(x)
        # Output layer
        # x = self.l_out1(x)
        x = relu(self.l_out1(x))
        # print("4    ", x.shape)
        x = relu(self.l_out2(x))

        
        return x
    

net = Net()
if use_cuda:
    net.cuda()
print(net)

Net(
  (lstm): LSTM(200, 32, num_layers=2, dropout=0.2)
  (l_out): Linear(in_features=32, out_features=200, bias=False)
)


### define criterion

In [ ]:
# Define a loss function and optimizer for this problem
# YOUR CODE HERE!
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.1)


In [ ]:
# Hyper-parameters
num_epochs = 50

# Initialize a new network
net = Net()

# Track loss
training_loss, validation_loss, accuracy = [], [],[]


# For each epoch
for i in range(num_epochs):
    
    # Track loss
    epoch_training_loss = 0
    epoch_validation_loss = 0
    
    net.eval()
        
    # For each sentence in validation set
    for inputs, targets in (x_val, y_val):
        print(inputs.shape, targets.shape)
        
        # Convert input to tensor
        inputs = torch.Tensor(inputs[:,np.newaxis,:,:].drop()) #[sequence,batch, input_size, channel]
        # inputs = inputs.permute(0, 2, 1)
        
        # Convert target to tensor
        targets = torch.LongTensor(targets[:,np.newaxis,:])
        
        # Forward pass

        outputs = net(inputs.cuda())
        
        # Compute loss

        loss = criterion(outputs, targets)
        # Update loss
        epoch_validation_loss += loss.detach().numpy()
    net.train()
    
    # For each sentence in training set
    for inputs, targets in (x_train, y_train):
        
        # Convert input to tensor
        inputs = torch.Tensor(inputs)
        # inputs = inputs.permute(0, 2, 1)
        
        # Convert target to tensor
        targets = torch.LongTensor(targets)
        
        # Forward pass
        outputs = net(inputs.cuda())
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass
        # zero grad, backward, step...
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Update loss
        epoch_training_loss += loss.detach().numpy()
        
    # Save loss for plot
    training_loss.append(epoch_training_loss/len(x_train))
    validation_loss.append(epoch_validation_loss/len(x_val))
    # Print loss every 5 epochs
    if i % 5 == 0:
        print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')

        
# Get first sentence in test set
inputs, targets = (x_test,y_test)[0]

# Convert input to tensor
inputs = torch.Tensor(inputs)
# inputs_one_hot = inputs_one_hot.permute(0, 2, 1)

# Convert target to tensor
targets = torch.LongTensor(targets)

# Forward pass
outputs = net.forward(inputs.cuda()).data.numpy()

#accuracy = accuracy_score(outputs, targets_idx.data.numpy())
#print("accuracu: ", accuracy)
print('\nInput sequence:')
print(inputs)

print('\nTarget sequence:')
print(targets)

print('\nPredicted sequence:')
print([np.argmax(output) for output in outputs])

# Plot training and validation loss
epoch = np.arange(len(training_loss))
plt.figure()
plt.plot(epoch, training_loss, 'r', label='Training loss',)
plt.plot(epoch, validation_loss, 'b', label='Validation loss')
plt.legend()
plt.xlabel('Epoch'), plt.ylabel('NLL')
plt.show()

(6, 200, 4) (8, 200, 4)


RuntimeError: ignored